In [1]:
import torch.optim as optim
import torch
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Capas de la red
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargar los datos de entrenamiento y prueba
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

def train_mnist(config):
    net = Net()
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"])
    train_loader = DataLoader(train_data, batch_size=int(config["batch_size"]), shuffle=True)

    for e in range(10):
        running_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            output = net(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        tune.report(loss=(running_loss / len(train_loader)))

def test_best_model(best_config):
    net = Net()
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(net.parameters(), lr=best_config["lr"])
    train_loader = DataLoader(train_data, batch_size=int(best_config["batch_size"]), shuffle=True)
    test_loader = DataLoader(test_data, batch_size=int(best_config["batch_size"]), shuffle=True)

    for e in range(10):
        running_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            output = net(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    test_loss = 0
    accuracy = 0
    with torch.no_grad():
        for images, labels in test_loader:
            log_ps = net(images)
            test_loss += criterion(log_ps, labels)
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
    print(f"Test loss: {test_loss/len(test_loader)}.. Test accuracy: {accuracy/len(test_loader)}")

search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([32, 64, 128])
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=20,
    grace_period=1,
    reduction_factor=2)

reporter = CLIReporter(
    metric_columns=["loss", "training_iteration"])

result = tune.run(
    train_mnist,
    resources_per_trial={"cpu": 1},
    config=search_space,
    num_samples=20,
    scheduler=scheduler,
    progress_reporter=reporter)

best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
test_best_model(best_trial.config)

2023-05-02 14:40:52,645	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-02 14:40:55,548	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-05-02 14:41:11,863	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (45 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


== Status ==
Current time: 2023-05-02 14:41:12 (running for 00:00:14.98)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 1.0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 16/20 (15 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+
| Trial name              | status   | loc             |   batch_size |          lr |
|-------------------------+----------+-----------------+--------------+-------------|
| train_mnist_a3484_00000 | RUNNING  | 127.0.0.1:10396 |           64 | 0.00098184  |
| train_mnist_a3484_00001 | PENDING  |                 |           32 | 0.0323804   |
| train_mnist_a3484_00002 | PENDING  |                 |           64 | 0.000184213 |
| train_mnist_a3484_00003 | PENDING  |                 |           32 | 0.0120016   |
| train_mnist_a3484

2023-05-02 14:42:15,901	WARNING util.py:244 -- The `start_trial` operation took 0.623 s, which may be a performance bottleneck.


== Status ==
Current time: 2023-05-02 14:42:15 (running for 00:01:18.68)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 3.0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 18/20 (15 PENDING, 3 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+
| Trial name              | status   | loc             |   batch_size |          lr |
|-------------------------+----------+-----------------+--------------+-------------|
| train_mnist_a3484_00000 | RUNNING  | 127.0.0.1:10396 |           64 | 0.00098184  |
| train_mnist_a3484_00001 | RUNNING  | 127.0.0.1:16172 |           32 | 0.0323804   |
| train_mnist_a3484_00002 | RUNNING  | 127.0.0.1:23716 |           64 | 0.000184213 |
| train_mnist_a3484_00003 | PENDING  |                 |           32 | 0.0120016   |
| train_mnist_a3484

2023-05-02 14:42:44,294	WARNING util.py:244 -- The `start_trial` operation took 0.578 s, which may be a performance bottleneck.


...

== Status ==
Current time: 2023-05-02 14:49:26 (running for 00:08:29.56)
Using AsyncHyperBand: num_stopped=15
Bracket: Iter 16.000: None | Iter 8.000: -0.05757234508666874 | Iter 4.000: -0.12202251341293255 | Iter 2.000: -0.23008149047729684 | Iter 1.000: -0.8310698084453784
Logical resource usage: 3.0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 20/20 (3 RUNNING, 17 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |      loss |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------|
| train_mnist_a3484_00015 | RUNNING    | 127.0.0.1:21736 |          128 | 0.0787008   | 0.0389307 |                    9 |
| train_mnist_a3484_00016 | RUNNING    | 127.0.0.1:23716 |    

== Status ==
Current time: 2023-05-02 14:49:46 (running for 00:08:49.53)
Using AsyncHyperBand: num_stopped=16
Bracket: Iter 16.000: None | Iter 8.000: -0.06243341922784845 | Iter 4.000: -0.12202251341293255 | Iter 2.000: -0.23008149047729684 | Iter 1.000: -0.8310698084453784
Logical resource usage: 1.0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 20/20 (1 RUNNING, 19 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |      loss |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------|
| train_mnist_a3484_00019 | RUNNING    | 127.0.0.1:10396 |           32 | 0.0505673   | 0.0199021 |                    7 |
| train_mnist_a3484_00000 | TERMINATED | 127.0.0.1:10396 |    

== Status ==
Current time: 2023-05-02 14:50:07 (running for 00:09:10.71)
Using AsyncHyperBand: num_stopped=16
Bracket: Iter 16.000: None | Iter 8.000: -0.053733514156674156 | Iter 4.000: -0.12202251341293255 | Iter 2.000: -0.23008149047729684 | Iter 1.000: -0.8310698084453784
Logical resource usage: 1.0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 20/20 (1 RUNNING, 19 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |      loss |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+-----------+----------------------|
| train_mnist_a3484_00019 | RUNNING    | 127.0.0.1:10396 |           32 | 0.0505673   | 0.0116972 |                    9 |
| train_mnist_a3484_00000 | TERMINATED | 127.0.0.1:10396 |   

2023-05-02 14:50:16,849	INFO tune.py:945 -- Total run time: 561.30 seconds (559.59 seconds for the tuning loop).


== Status ==
Current time: 2023-05-02 14:50:16 (running for 00:09:19.61)
Using AsyncHyperBand: num_stopped=16
Bracket: Iter 16.000: None | Iter 8.000: -0.053733514156674156 | Iter 4.000: -0.12202251341293255 | Iter 2.000: -0.23008149047729684 | Iter 1.000: -0.8310698084453784
Logical resource usage: 0/8 CPUs, 0/1 GPUs
Result logdir: C:\Users\Gollo\ray_results\train_mnist_2023-05-02_14-40-56
Number of trials: 20/20 (20 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+------------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |       loss |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+------------+----------------------|
| train_mnist_a3484_00000 | TERMINATED | 127.0.0.1:10396 |           64 | 0.00098184  | 1.52598    |                    1 |
| train_mnist_a3484_00001 | TERMINATED | 127.0.0.1:16172 |           3